# Intro aux index

## SQL DB vs Index
Les index sont des cousins des bases de données (DB - databse) dans le sens où ils stockent de la données et permettent d'y accéder via des requêtes. Commençons par voir les index comme des DB munis d'une seule table qui contient des **documents** qu'un utilisateur voudra requêter. Les index ne sont pas capables de jointure et les opérations classiques du SQL y sont difficiles (`GROUP BY`) voire simplement impossibles (`JOIN`, `ROW NUMBER`). Appelons *corpus* l'ensemble de documents indéxés.

NB: dans une moindre mesure, les *index* sont régulièrement utilisés dans les coulisses des DB SQL sur les clefs fréquemment manipulés. On y reviendra.

### Alors à quoi sert un index ?

Un **index** est un puissant outil *pour retrouver des documents à partir de **query** sur leurs attributs, une partie de leurs attributs, ou d'une information partielle sur leurs attributs*. De plus, ces outils sont souvent équipés d'un système de **scores d'adéquation** qui *ordonnent* les documents par ordre de pertinence face à la query.

Il existe une infinité de façon de définir un score d'adéquation. En effet, il ne s'agit "que" d'un calcul opéré sur le coupe `(query, document)` et qui propose une mesure de l'adéquation query/document. Nous parlerons simplement de *score* par la suite

# Index lexical pour la fouille de texte

Au delà du simple matching d'attributs, un index lexical est capable de retrouver efficacement des documents qui contiennent les mots de la query. Les index sont équipés de structures de données complexes permettant d'insérer des documents et d'y effectuer des recherches en complexité $O(\log n)$ (où $n$ est le nombre de documents). Contrairement à une `hashMap`, il ne s'agit pas de simplement `get` un document via son identifiant (auquel cas il s'agit d'une compléxité de $O(1)$) mais de trouver *tous les documents qui correspondent à une recherche*. 

## Exercice préliminaire
But: mimer les index et leur capacité à retrouver tous les documents qui parlent d'un mot. 

Proposer :
- une structure de donnée optimisée pour représenter les documents et leur texte (on considèrera uniquement les `name`, `description_beer`, `description_brewery`)
- une fonction qui utilise cette structure pour trouver le plus rapidement possible tous les documents qui possède le mot "scottish"

Le dataset `/datasets/beers_full.csv` contient un condensé des infos de chaque bière. Le dictionnaire `beer_id2desct` associe à chaque `beer_id` sa description

In [92]:
import pandas as pd
df = pd.read_csv("/datasets/beers_full.csv")
beer_id2desct = df.fillna("").apply(lambda row: " ".join([str(row["name"]), row["description_beer"], row["description_brewery"]]), axis=1).to_dict()

print("Beer id 42:", beer_id2desct[42])

Beer id 42: Maracaibo Especial A rich brown ale inspired by the enigmatic monastic brews of Belgium, and the mysterious mist shrouded jungles of the tropics. Brewed with real cacao, and spiced with cinnamon and sweet orange peel for a sensual delight. A brew to be sipped, savored, and enjoyed! Welcome to a land of open fermentation, oak barrel aging, and bottle conditioning. At Jolly Pumpkin Artisan Ales we are dedicated to more than the traditions of old world craftsmanship. Everything we do is designed to create ales of outstanding art and flavor. Focusing on traditional rustic country style beers brought to life through labor and love, we strive to create beers to lighten the spirit, and soothe the soul. Sharing our joy to the betterment of mankind is the most that we could hope for.


Chacun testera la vitesse de son code avec le bloc suivant:
```python
%%timeit
for word in ["ale", "scottish", "Bouffay"]:
    beer_ids = fetch_doc_ids_having_word(word)
```
La "magic function" `%%timeit` mesure le temps moyen d'éxecution de la cellule ne l'appelant ~1k-1M fois.

Perf à battre pour cet exo 😎:

In [93]:
def fetch_doc_ids_having_word(word):
    resp = []
    for id_, descr in beer_id2desct.items():
        if word in descr:
            resp.append(id_)
    return resp

In [94]:
%%timeit
for word in ["ale", "scottish", "Bouffay"]:
    beer_ids = fetch_doc_ids_having_word(word.lower())

4.74 ms ± 31.9 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [95]:
%%time
# get vocabulary
vocab = sorted(set([word for doc in beer_id2desct.values() for word in doc.lower().split()]))

CPU times: user 60.9 ms, sys: 4.02 ms, total: 64.9 ms
Wall time: 64 ms


In [96]:
%%time
# compute posting list in a naïve whay
posting_list = {
    word: {
        doc_id: [i for i, w in enumerate(str(doc_str).lower().split()) if w == word]
        for doc_id, doc_str in beer_id2desct.items()
        if " " + word + " " in str(doc_str).lower()
    }
    for word in vocab
}


def fetch_doc_ids_having_word(word):
    return posting_list[word.lower()]

CPU times: user 2min 12s, sys: 30.9 ms, total: 2min 12s
Wall time: 2min 12s


In [97]:
%%timeit
for word in ["ale", "scottish", "Bouffay"]:
    beer_ids = fetch_doc_ids_having_word(word.lower())

655 ns ± 7.84 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [103]:
beer_ids = fetch_doc_ids_having_word("scottish".lower())

In [104]:
beer_ids

{110: [3],
 217: [31, 51],
 292: [2],
 676: [1],
 990: [1],
 1047: [3],
 1977: [3],
 2002: [2],
 2045: [2],
 2048: [1],
 2064: [2],
 2070: [2],
 2269: [0, 11, 34],
 2396: [23],
 2513: [2],
 2518: [1],
 2585: [12],
 2818: [2],
 3051: [4],
 3362: [1],
 3836: [2],
 4119: [11, 65],
 4146: [47],
 4266: [18],
 4286: [123],
 4496: [1, 21, 78],
 4664: [6, 20],
 4673: [14, 29],
 4774: [3],
 4791: [6],
 4831: [59],
 5005: [28],
 5035: [1],
 5325: [58],
 5614: [35],
 5748: [32],
 5755: [19],
 5758: [73],
 5843: [27]}

## Structure de donné de l'index lexical inversé

Un index maintient une ["posting list" ou "index inversé"](https://www.wikiwand.com/en/articles/Inverted_index) en mémoire : à la façon d'un glossaire dans un livre, l'index maintient en mémoire un vocabulaire (les mots contenus dans tout le corpus) ainsi que, pour chaque mot, **la liste de tous les documents qui l'évoquent ainsi que les positions du mot dans les documents**. Trouver les documents qui contiennent une série de mot devient rapide : $O(\log n)$.

Exemple :
Soit le (mini) corpus suivant
- "portez ce vieux whiskey au juge blond qui fume"
- "On fume ce type de malts pour obtenir ce whiskey tourbé"

Calculons la `posting_list`

In [78]:
beer_id2desct = {
    1: "portez ce vieux whiskey au juge blond qui fume",
    2: "On fume ce malt pour obtenir ce whiskey tourbé",
    3: "La fermentation haute donne des bières plus fruitées.", 
    4: "Ce houblon ajoute une amertume distincte à la bière.", 
    5: "Le malt apporte la rondeur à la bière mais un malt torréfié apporte des arômes de café à la bière.", 
    6: "La bière de fermentation basse est plus légère et rafraîchissante.", 
    7: "La température de fermentation influence le goût final de la bière.", 
}

## BM25 : score de priorité pour un index lexical
Le score de référence dans le domaine de la recherche lexicale est le [BM25](https://www.wikiwand.com/fr/articles/Okapi_BM25) (pas nécessaire de comprendre les formules). En première approximation, supposons que ce score attribué à chaque document calcule, pour chaque mot de la query, la fréquence d'apparition du mot dans le document pondéré par des notions de rareté du mot dans le corpus (une `posting_list` permet d'obtenir cette information très facilement).



In [107]:
from bm25s import BM25, tokenize

# Tokenize the corpus and index it
corpus_tokens = tokenize(list(beer_id2desct.values()))
retriever = BM25(corpus=list(beer_id2desct.values()))
retriever.index(corpus_tokens)

retriever.retrieve(tokenize("youoi"), k=3)

Results(documents=array([['Hocus Pocus Our take on a classic summer ale.  A toast to weeds, rays, and summer haze.  A light, crisp ale for mowing lawns, hitting lazy fly balls, and communing with nature, Hocus Pocus is offered up as a summer sacrifice to clodless days. Its malty sweetness finishes tart and crisp and is best apprediated with a wedge of orange. Burlington microbrewers of Humble Patience, Fat Angel, #9, Blind Faith IPA, and Heart of Darkness Oatmeal Stout.',
        'Ambr  La Brasserie du Bouffay est situ', 'Abhi beer  ']],
      dtype='<U447'), scores=array([[0., 0., 0.]], dtype=float32))

## Index Vespa
[Vespa](https://vespa.ai/) est un index moderne, puissant, hautement distribué, concurrent du très connu [ElasticSearch](https://www.elastic.co/fr/). Ces 2 index permettent d'être utilisés en mode lexical pour de la recherche d'information dans un corpus.

Nous utiliserons Vespa pendant ce TP : ne pas hésiter à [aller voir la doc](https://docs.vespa.ai/en/overview.html) (attention, ne pas se laisser impressionner par toute la technique) ou poser des questions à leur [chat-bot](https://search.vespa.ai/) (même remarque).

Une instance Vespa tourne sur le serveur à l'adresse http://vespa:8080 . Pour y accéder facilement nous utiliserons le package Python PyVespa (voir [le doc](https://pyvespa.readthedocs.io/en/latest/index.html)) proposé par l'équipe Vespa :

In [108]:
from vespa.application import Vespa, VespaSync
import json

vespa = Vespa(url="http://vespa", port=8080)
vespa.wait_for_application_up(30)

Application is up!


## Recherche distribuée
Pour stocker 10G documents, un seul serveur ne suffit pas. Vespa (comme ES) utilise un cluster de machine, chacune possédant un fragment (shard) du corpus. Des options de redondance existent : une donnée est stockée sur plusieurs noeuds. Cette redondance permet de perdre des noeuds sans pour autant perdre la moindre data.

Distribution implique ici parallélisme : lors d'une requête, chaque noeud effectue la plupart des étapes de recherche de son côté. Les résultats ne sont fusionnés que très tardivement pour tirer profit du parallélisme jusqu'en dernière minute.

## Multi-stage retrieval
Vespa est capable d'indéxer des milliards de documents (exemple avec [Vinted](https://vinted.engineering/2024/09/05/goodbye-elasticsearch-hello-vespa/) ou [Spotify](https://engineering.atspotify.com/2022/03/introducing-natural-language-search-for-podcast-episodes/) ou Qwant).

Pour sortir le top10 résultats pour une query parmi 10G documents, plusieurs étapes sont nécessaires. La recherche est un entonnoir à data qui s'effectue indépendamment sur chaque noeud:
1. matching: ne récupérer que les documents qui parlent dess mots de la query (via posting list)
2. first-phase: classer grossièrement les documents survivants sur des critères d'adéquation query/document faciles à calculer (BM25 via posting list) et ne garder que le top1k par noeud
3. second-phase: classer finement les 1k documents survivants sur chaque noeud afin de préparer le top final. Ne prendre que le top100 par noeud
4. fusion: tous les noeuds acheminent leurs top100 respectifs à un noeud principal qui crée le résultat final

## Exemple de requête à Vespa
En utilisant le client Python PyVespa:

In [111]:
# requête simple
resp = vespa.query(
    {
        "yql": "select * from beer where userQuery()", # <-- on dirait du SQL ! Mais en plus limité ...
        "hits": 2, # <-- nombre de résultats voulus
        "query": "stout", # <-- query textuelle, nous verrons son usage après
        "presentation.summary": "textual"
    }
)
print("Réponse de Vespa:\n")
print(json.dumps(resp.json["root"], indent=2))

Réponse de Vespa:

{
  "id": "toplevel",
  "relevance": 1.0,
  "fields": {
    "totalCount": 557
  },
  "coverage": {
    "coverage": 100,
    "documents": 5699,
    "full": true,
    "nodes": 1,
    "results": 1,
    "resultsFull": 1
  },
  "children": [
    {
      "id": "index:beer_content/0/a6a767bbba7d4e24424f25fa",
      "relevance": 7.0372892428405835,
      "source": "beer_content",
      "fields": {
        "sddocname": "beer",
        "name": "Kalamazoo Stout",
        "id": "4134",
        "brewery": "Bell's Brewery Inc.",
        "description_beer": "A full-bodied stout with plenty of roast flavor. Kalamazoo Stout is available year round, leading our vast portfolio of stouts."
      }
    },
    {
      "id": "index:beer_content/0/21ce68912bc0dda9d1677ec8",
      "relevance": 6.910275544426504,
      "source": "beer_content",
      "fields": {
        "sddocname": "beer",
        "name": "Oatmeal Stout",
        "id": "3751",
        "brewery": "Troegs Brewing",
        "de

Un index (Vespa ou ES) a besoin de connaître le schéma de data à indexer. Il s'agit de :
- l'ensemble des attributs d'un document à indexer : nom, type (string, float, ...)
- si nécessaire, la façon dont un champ doit être utilisés par Vespa lors de l'indexation : simple attribut, indexé par `posting list`, exploitable via BM25, affichable dans les réponses aux queries, etc ...

Elastic nomme cela [mapping](https://www.elastic.co/guide/en/elasticsearch/reference/current/mapping.html), Vespa nomme cela la [schema](https://docs.vespa.ai/en/schemas.html). Exemple avec nos données montées sur Vespa
```
schema beer {
    document beer {
        field id type string {
        }
        field name type string {
            indexing: index | summary
        }
        field description_beer type string {
            indexing: index | summary | attribute
            index: enable-bm25
        }
        field brewery type string {
            indexing: index | summary
            index: enable-bm25
        }
        field address1 type string {
            indexing: index | summary | attribute
        }
        field description_brewery type string {
            indexing: index
            index: enable-bm25
        }
    [...]
    }

  [...]

}
```

Vespa est très orienté recherche d'information et permet également de définir des façons de très précise le **score d'adéquation** à utiliser pour classer (ranker) les documents. Il s'agit d'une autre partie du schéma :
```
  [...]

    rank-profile basic_bm25 {
        first-phase {
            expression {
                bm25(name) + bm25(description_beer)
            }
        }
    }

  [...]
```

# Uses cases

## UC-1 : description data

Attention, plus compliqué qu'en SQL. Voir les docs spécifiques
- [doc spécifique Vespa sur le grouping](https://docs.vespa.ai/en/grouping.html)
- [doc sur le SQL façon Vespa - YQL](https://docs.vespa.ai/en/vespa-cli#cheat-sheet)

**Ne pas chercher à aller jusqu'au bout !!** Nous remarquerons assez vite que c'est galère avec Vespa

- Q1: Combien y a-t-il de bières dans la DB ?
- Q2: Top10 brasseries les plus représentées avec le nombre de bière par brasserie ?
- Q3: Top10 des bières les plus fortes (ABV) en France ?
- Q4: Par pays, nombre de brasseries qui proposent des bières de type `Porter` et ABV moyen de celles-ci ?
- Q5: Mediane du nombre de bière par pays ?

In [115]:
# Q1
resp = vespa.query(
    {
        "yql": "select * from beer where true limit 0 | all( output(count()) )", 
        "hits": 10
    }
)
print(json.dumps(resp.json["root"]["children"], indent=2))

[
  {
    "id": "group:root:0",
    "relevance": 1.0,
    "continuation": {
      "this": ""
    },
    "fields": {
      "count()": 5699
    }
  }
]


In [124]:
# Q2: Top10 brasseries les plus représentées avec le nombre de bière par brasserie ?
resp = vespa.query(
   {
       "yql": "select * from beer where true limit 0 | all( group(brewery) order(-count()) each(output(count()))) ",
       "hits": 1
   } 
)
print(json.dumps(resp.json["root"], indent=2))

{
  "id": "toplevel",
  "relevance": 1.0,
  "fields": {
    "totalCount": 5699
  },
  "coverage": {
    "coverage": 100,
    "documents": 5699,
    "full": true,
    "nodes": 1,
    "results": 1,
    "resultsFull": 1
  },
  "children": [
    {
      "id": "group:root:0",
      "relevance": 1.0,
      "continuation": {
        "this": ""
      },
      "children": [
        {
          "id": "grouplist:brewery",
          "relevance": 1.0,
          "label": "brewery",
          "continuation": {
            "next": "BGAAABECBEBC"
          },
          "children": [
            {
              "id": "group:string:Midnight Sun Brewing Co.",
              "relevance": 1.0,
              "value": "Midnight Sun Brewing Co.",
              "fields": {
                "count()": 57
              }
            },
            {
              "id": "group:string:Rogue Ales",
              "relevance": 0.9,
              "value": "Rogue Ales",
              "fields": {
                "count()":

In [130]:
#Q3: Top10 des bières les plus fortes (ABV) en France ?
resp = vespa.query(
   {
        "yql": "select * from beer where true and country contains 'France' order by abv desc",
        "presentation.summary": "textual",
        "hits": 10
   } 
)
print(json.dumps(resp.json["root"], indent=2))

{
  "id": "toplevel",
  "relevance": 1.0,
  "fields": {
    "totalCount": 18
  },
  "coverage": {
    "coverage": 100,
    "documents": 5699,
    "full": true,
    "nodes": 1,
    "results": 1,
    "resultsFull": 1
  },
  "children": [
    {
      "id": "index:beer_content/0/934b5358a7f53b68b8e76113",
      "relevance": 0.0,
      "source": "beer_content",
      "fields": {
        "sddocname": "beer",
        "name": "Belzebuth",
        "id": "2222",
        "brewery": "Brasserie Grain D'Orge",
        "description_beer": "Beer of top fermentation, Belzebuth represents the achievement of a more than one hundred year-old know-how, the one of the Brewery Grain d'Orge. Under its copper colour, it hides a strong character for the lovers of sensations. It owes its strength to the subtle mixture of pure malts, hops and yeasts especially selected by our Master-Brewer.\r\n\r\nAt one time this beer was 15%. After the name change it was lowered to 13%."
      }
    },
    {
      "id": "index:

In [113]:
# your code

# UC-2 : préparer un dataset de ranking 

Plusieurs problèmes pour résoudre ce use-case:
- cf intro : un index ne peut pas faire de jointure 
- cf UC-1 : les grouping sont horribles à réaliser

Conclusion : les index ne sont vraiment pas adaptés pour fabriquer des datasets !

In [113]:
# your code

# UC-3 : récupérer les docs qui parlent d'un mot

Peut-on utiliser Vespa pour réaliser un mini moteur de recherche ? 

**Rappel:** une configuration Vespa intègre la définition d'un **score d'adéquation** entre query et document ! Le `rank-profile` suivant existe sur l'instance Vespa utilisées :
```
  [...]

    rank-profile rank-brewery-and-descr inherits root{
        first-phase {
            expression {
                bm25(description_brewery) + bm25(description_beer)
            }
        }
    }

  [...]
```

**Question 1:** expliquer en 2 phrases comment le rank-profile `rank-brewery-and-descr` va agir.

**Question 2:** pour différentes requêtes textuelles très simples à base de mot-clef, retrouver les bières qui semblent répondre à la demande :
- trouver les bières ou les brasseries qui parlent de bières "fine"
- idem pour "juicy"
- idem pour "genuine"
- idem pour les bières mâturées dans des "oak cask" (fûts en chêne) -> combien y en a-t-il ? $N_1$
   - idem pour les bières qui évoquent uniquement "cask" -> combien y en a-t-il ? $N_{1,1}$
   - idem pour celles ne parlant que de "oak" -> combien y en a-t-il ? $N_{1,2}$
- idem pour les bières qui évoquent "oak" et "cask" -> combien y en a-t-il ? $N_{2}$

In [147]:
# requête simple
QUERY = "cask"
resp = vespa.query(
    {
        "yql": "select * from beer where userQuery()", 
        "hits": 2, # <-- nombre de résultats voulus
        "query": QUERY,
        "presentation.summary": "textual"
    }
)
print("Réponse de Vespa:\n")
print(json.dumps(resp.json["root"], indent=2))

Réponse de Vespa:

{
  "id": "toplevel",
  "relevance": 1.0,
  "fields": {
    "totalCount": 62
  },
  "coverage": {
    "coverage": 100,
    "documents": 5699,
    "full": true,
    "nodes": 1,
    "results": 1,
    "resultsFull": 1
  },
  "children": [
    {
      "id": "index:beer_content/0/582967e04a0cf6bf51211863",
      "relevance": 11.529384358323982,
      "source": "beer_content",
      "fields": {
        "sddocname": "beer",
        "name": "Arthur's Nugget Pale Ale",
        "id": "3852",
        "brewery": "Otto's Pub and Brewery",
        "description_beer": "A cask-conditioned American Pale Ale. This is a bright dry ale brewed only with American Nugget hops.",
        "description_brewery": "Otto's typically offers a selection of 9-11 beers on draft. These include seasonal offerings as well as some of our favorite year-round ales and lagers including our special cask-conditioned ales. All of our beers are brewed on-premises at Otto's for the utmost freshness and quality 

In [113]:
# your code

# UC-4 : vectorisation des description des bières

## Index et "scrolling/visiting"

Les index ne sont "pas faits" pour manipuler la donnée qu'ils hébergent, néanmoins il existe plusieurs systèmes qui permettent à un code arbitraire de s'exécuter sur chaque document. Vespa nomme son système `visitor` ([doc](https://docs.vespa.ai/en/visiting.html)) et permet de 
- selectionner des documents à processer
- dumper l'index en parallèle (et possiblement les modifier + update à la volée)
- appliquer un code arbitraire à des documents (Nécessite le recours à Java pour expliciter l'opération :o )

L'équivalent Elastic Search est le [scroll](https://www.elastic.co/guide/en/elasticsearch/guide/current/scroll.html).

**Remarque:** on comprend vite qu'il n'est pas forcément simple de manipuler une donnée une fois qu'elle est indexée. Vespa étant très tourné vers le ML, il est toutefois possible de vectoriser des documents automatiquement lors de leur upsert. Voir la [doc Vespa sur la gestion en direct des embeddings](https://docs.vespa.ai/en/embedding.html#huggingface-embedder)

## Vespa est capable de vectorisation à la volée

Vespa (contrairement à ES) est capable de gérer nativement des embedding, de créer des vecteurs à la volée à partir des documents indéxés, de les retrouver dans des index vectoriels (explications détaillées au prochain TP) ..! Voir [ce blog post](https://blog.vespa.ai/combining-matryoshka-with-binary-quantization-using-embedder/) de Vespa sur le sujet.

In [149]:
# requête simple
resp = vespa.query(
    {
        "yql": "select * from beer where userQuery()", 
        "hits": 1, # <-- nombre de résultats voulus
        "query": "stout",
        "presentation.summary": "textual"
    }
)
print("Réponse de Vespa:\n")
print(json.dumps(resp.json["root"], indent=2))

Réponse de Vespa:

{
  "id": "toplevel",
  "relevance": 1.0,
  "fields": {
    "totalCount": 557
  },
  "coverage": {
    "coverage": 100,
    "documents": 5699,
    "full": true,
    "nodes": 1,
    "results": 1,
    "resultsFull": 1
  },
  "children": [
    {
      "id": "index:beer_content/0/db8419f40e4efc774fdce482",
      "relevance": 6.78948031198024,
      "source": "beer_content",
      "fields": {
        "sddocname": "beer",
        "name": "Coopers Best Extra Stout",
        "id": "4933",
        "brewery": "Coopers Brewery",
        "description_beer": "Now here's a beer with punch! \r\n\r\nCoopers Best Extra Stout is a beacon for lovers of a hearty brew. With its robust flavour it is everything a stout should be. \r\n\r\nBrewed naturally using a top fermentation method, Coopers Stout's unique rich, dark texture comes from specially roasted black malt. \r\n\r\nCoopers Best Extra Stout contains no additives and no preservatives.",
        "description_brewery": "Coopers Brew

# UC-5 : answer question in corpa
**Grandes lignes :** trouvons les documents qui répondent à une question. Exemple : à partir de la description vectorisée à UC-4 pour chaque bière, comment trouver les bières qui répondent à une description plus complète ? Exemple:
- "very bitter beer with smoky taste"
- "fruity sour - balanced sourness"
- "weird beer"


In [154]:
query = "child friendly beer"
resp = vespa.query(
    {
        "yql": "select * from beer where {targetHits:3}nearestNeighbor(mrl_embedding, q)",
        "input.query(q)": "embed(mxbai, @text)",
        "ranking.profile": "ann",
        "text": query,
        "presentation.summary": "textual"
    }
)
print(json.dumps(resp.json["root"]["children"], indent=2))

[
  {
    "id": "index:beer_content/0/805163a06d020be96e66db87",
    "relevance": 0.7683171359270097,
    "source": "beer_content",
    "fields": {
      "sddocname": "beer",
      "name": "Colonel Blide's Cask Ale",
      "id": "5852",
      "brewery": "Cricket Hill",
      "description_beer": "A tasty, moderately hoppy, easy to drink beer with an orange marmalade hue with a frothy, off-white head. An herbal hop aroma with a hint of lemon and pine. This beer is not overly bitter, with orangy citrus and pine hop with slight earthy notes! There is a malty biscuit flavor that balances the hops. Makes me wish I had some Fish & Chips!!",
      "description_brewery": "We believe that New Jersey is starting to learn what the rest of the country already knows; that local microbrews are some of the best beers in the world! Finally, curious beer drinkers are stepping out from under the mind-numbing barrage of large brewery advertising and deciding for themselves what they like. Our beers are de

## Index vectoriel : NN et ANN, équivalent ML des index lexicaux
Depuis ~2020, le ML permet de représenter des documents textuels par des vecteurs en grande dimension (appelés **embedding**) qui possèdent l'énorme propriété de traduire numériquement/vectoriellement l'information sémantique contenue dans les documents. De surcroit, ces embeddings peuvent se comparer algébriquement très simplement dans le sens où 2 embeddings "proches" (dans leur espace) correspondent à des objets proches (dans notre perception). Le uses-case 4 des TP précédent visait à obtenir de tels embeddings à partir d'un service externe.

et les algorithmes *Nearest Neighbors* ou plus récemment *Approximated Nearest Neighbors*.
### Recherche par plus proche voisins 
#### Algorithme Nearest Neighbors - NN

Puisqu'il est possible de représenter (presque) tout document sous format embedding et que ces derniers ont la propriété d'être comparables entre eux, un nouveau type de recherche s'ouvre : recherche par embeddings les plus proche de l'embedding d'une query. Opérer une recherche à partir d'une query revient à trouver les *plus proches voisins* (Nearest Neighbors - NN) de l'embedding de la query parmi les embeddings de documents.

Exemple en dimension 2 : les coordonnées GPS 2D d'une ville sont en quelque sort un embedding basique d'une ville. Trouver les 5 villes les plus proches de Nancy est simple : il suffit de cacluler les distances de Nancy à toutes les villes grâce à leurs coordonnées et de trouver les top5 distances les plus faibles.

**Problème:** une telle recherche exhaustive implique $O(n^2)$ calculs où $n$ est le nombre de villes. Si $n=10^6$, le calcul devient difficile.

#### Variante Approximative Nearest Neihbors - ANN

**Solution proposée:** sachant qu'il est inutile de calculer la distance entre Nancy et Timbuktu ou New-York (celles-ci ne seront jamais dans le top5 proximité), il peut être intéressant de restreindre le champ de recherche afin de ne payer une recherche exhaustive en $O(n^2)$ que pour une poignée de villes qu'on sait déjà être "proches". Dans notre exemple, une recherche limitée au département de la ville cible et aux départements limitrophes suffit. 

Il s'agit d'un début de compréhension de la famille d'algorithmes Approximative Nearest Neihbors (ANN par la suite) qui permet de casser la complexité du problème de recherche de plus proches voisins en hierarchisant l'information. Cette hierarchisation se fait via une structure de donnée particulière ; l'implémentation la plus courante en 2024 est [Hierarchical Navigable Small World - HNSW](https://www.wikiwand.com/en/articles/HNSW_indexes).

Remarque : l'algo qui traduit réellement la hierarchisation stricte est plutôt de la famille [K-d tree](https://www.wikiwand.com/en/articles/K-d_tree) mais il est inefficace pour des vecteurs de dimension $k=768+$ comme c'est le cas pour la plupart des embeddings